**Condor Instruments** - Complete sleep analysis demonstration

*Julius A. P. P. de Paula*

*jp@condorinst.com.br* 



*   If you would like, you can access a Google Colab version of this page at: https://drive.google.com/file/d/1KeoceVsCIn5tKk-8XAMIMvmgCSiC5Chf/view?usp=sharing



1) Package installation and upgrade

In [ ]:
## we already have them installed :D

# !pip install wget # installs library for file download
# !pip install xgboost --upgrade # upgrades package used in offwrist algorithm

2) Importing packages

In [ ]:
# these packages are required for obtaining the path to the current file
import sys
import inspect
import os
root = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) # path to "this" directory

In [ ]:
import numpy as np # mathmatics library
import pandas as pd # data science library

In [ ]:
## we already have all the dependencies locally :D

# # dependency download
# import wget
# URL = "https://github.com/Condor-Instruments/actigraphy-tutorials-sample/blob/master/demo-dependencies.zip?raw=true"
# response = wget.download(URL, "demo-dependencies.zip")

# # file unzip
# import zipfile
# with zipfile.ZipFile("demo-dependencies.zip", 'r') as zip_ref:
#   zip_ref.extractall(root)

from cspd_wrapper import cspd_wrapper # algorithm for main sleep period detection

from nap_wrapper import nap_wrapper # algorithm for secondary sleep period detection

from logread import LogRead as lr # class for log file reading

from boosting_offwrist_wrapper import boosting_offwrist_wrapper # algorithm for offwrist detection

from colekripke import ColeKripke as ck # algorithm for WASO detection

from nights_df import nights_df # helper algorithm for daily processing

from simple_actogram import actigraphy_single_plot_actogram

3) Reading files

For this demonstration we've made 3 files available: input0.txt, input1.txt and input2.txt

In [ ]:
file = "input1.txt" # file subject to analysis

df = lr(file).data # with LogRead class the file is read to a DataFrame from pandas library
npindex = df.index.to_numpy()
df = df[pd.Timestamp(npindex[0]):pd.Timestamp(npindex[int(len(npindex)/3)])]

4) Preparing the input DataFrame

In [ ]:
# state-related columns will be separated for better visuallization, all of them will be initially filled with zeros
df["state"] = np.zeros(len(df))
df["offwrist"] = np.zeros(len(df)) 
df["sleep"] = np.zeros(len(df)) 

int_temp = df["TEMPERATURE"].to_numpy()
ext_temp = df["EXT TEMPERATURE"].to_numpy()
int_temp = np.where(int_temp > 0, int_temp, 0)
ext_temp = np.where(ext_temp > 0, ext_temp, 0)
int_temp = np.where(int_temp < 42 , int_temp, 42)
ext_temp = np.where(ext_temp < 42 , ext_temp, 42)
df["int_temp"] = int_temp
df["ext_temp"] = ext_temp

# pre-scaled temperatures will be used for plotting
scale = np.max([np.max(ext_temp),np.max(int_temp)])
df["int_temp_"] = int_temp/scale
df["ext_temp_"] = ext_temp/scale

print(df)

5) Offwrist

The algorithm for offwrist period detection is meant to filter out of the analysis the moments when the subject is not wearing the actigraph. It is based on the Gradient Boosting algorithm provided by the XGBoost library. With the activity measure PIM and internal and external temperature, new auxiliary variables are computed and all are fed to the algorithm to generate a classification.

In [ ]:
out = boosting_offwrist_wrapper(df) # offwrist detection

# column updates
df["state"] = out
df["offwrist"] = 0.25*out 

# we'll use actograms for visuallizing the data
fig = actigraphy_single_plot_actogram(df, ["PIM", "int_temp_","ext_temp_","sleep","offwrist"], [False, True, True, True, True], 12, dt = "DATE/TIME")
fig.show()

6) Main sleep periods

The algorithm for main sleep period detection is based on an implementation of the Crespo algorithm that was initially described in the scientific literature by Crespo et al in 2012. The algorithm consists on a delimitation of the periods of high and low activity inside the time series through a percentile-based thresholding operation. After this initial delimitation, a refinement procedure takes place using metrics that we've developed.

In [ ]:
# columns need to renamed before being fed to the algorithms
df = df.rename(columns={"DATE/TIME":"datetime",
                        "PIM":"activity",
                        "TEMPERATURE":"int_temp",
                        "EXT TEMPERATURE":"ext_temp"})

out = cspd_wrapper(df) # main sleep period detection (bed time and getup time)

df["state"] = out
df["sleep"] = np.where(out == 4,0,out)

fig = actigraphy_single_plot_actogram(df, ["activity", "int_temp_","ext_temp_","sleep","offwrist"], [False, True, True, True, True], 12, dt = "datetime")
fig.show()

7) Secondary sleep periods (Naps)

The algorithm used for nap detection is the same Crespo algorithm described above but with different parameters. The same principle is used, we aim to detect periods of low movement, but we'll search only the periods defined as Awake previously and make the algorithm more sensitive to smaller variations in movement amplitude to find these relatively short sleep periods. The same is valid during the refinement stage of the algorithm, we'll feed in different parameters, specific to this particular problem.

---



In [ ]:
# some of the parameters that differentiate our implementations relate to the length of the sleep period we wish to detect,
# we seek to find short periods, and some are the inputs to the refinement logic. the refinement step improves the boundaries of the sleep period

out = nap_wrapper(df) # secondary sleep period detection (nap bed time and getup time)

df["state"] = out
df["sleep"] = np.where(out == 7,1,df["sleep"].to_numpy())

fig = actigraphy_single_plot_actogram(df, ["activity", "int_temp_","ext_temp_","sleep","offwrist"], [False, True, True, True, True], 12, dt = "datetime")
fig.show()

8) WASO

The Wakefullness After Sleep Onset detection uses our implementation of an algorithm described in the scientific literature by Cole et al in 1992. It consists on a weighted sum rolling window operation followed by a thresholding operation. In our implementation we use a different window size and weights, we choose to do so based on the results we got from optimization studies carried with AI and other statistical techniques.

In [ ]:
onwrist = np.where(out == 4, False, True) # actigraph on the wrist mask

# input variables read only in the offwrist periods
stamps = df["datetime"].to_numpy()[onwrist]
zcm = df["ZCMn"].to_numpy()[onwrist]

n = len(zcm) # time-series length
state = np.zeros(n) # auxiliary array to compute new states 

in_bed = out[onwrist] # this array acts as a sleep journal, we get the information of whether or not the subject is in bed
nights = nights_df(stamps,in_bed,wake_thresh=60,search_gap=False) # night segregation

num_nights = len(nights) # number of nights present in the time series

# nightly sleep statistics arrays
waso = np.nan*np.ones(num_nights)
tbt = np.nan*np.ones(num_nights)
tst = np.nan*np.ones(num_nights)
sol = np.nan*np.ones(num_nights)
soi = np.nan*np.ones(num_nights)
nw = np.nan*np.ones(num_nights)
eff = np.nan*np.ones(num_nights)
bts = []
gts = []

for i in range(num_nights):
    bt = nights.at[i,"bt"] # bed time index
    gt = nights.at[i,"gt"] # getup time index

    bts.append(stamps[bt]) # bed time 
    gts.append(stamps[gt]) # getup time

    input = zcm[bt:gt]
    cole = ck(input, # WASO computations are carried nightly
              P=0.000464,
              weights_before=[34.5,133,529,375,408,400.5,1074,2048.5,2424.5],
              weights_after=[1920,149.5,257.5,125,111.5,120,69,40.5],
              )
    cole.model(np.zeros(gt-bt))

    cpred = cole.filtered_weighted # states on the current night
    
    # SOL computation
    latency = 0
    while cpred[latency] > 0:
        latency += 1

    # SOI computation
    innertia = len(cpred)-1
    while cpred[innertia] > 0:
        innertia -= 1

    # Computing the number of wake periods during the night
    edges = np.diff(cpred)
    num_awake = np.sum(np.where(edges>0,1,0))

    sol[i] = latency
    soi[i] = len(cpred)-1-innertia
    waso[i] = np.sum(cpred[latency:innertia])
    nw[i] = num_awake
    tbt[i] = gt-bt
    tst[i] = tbt[i]-waso[i]-soi[i]-sol[i]
    eff[i] = tst[i]/tbt[i]
    
    state[bt:gt] = 1-cpred

nights["tbt"] = tbt
nights["waso"] = waso
nights["sol"] = sol
nights["soi"] = soi
nights["tst"] = tst
nights["nw"] = nw
nights["eff"] = eff

nights.insert(0,"gts",gts)
nights.insert(0,"bts",bts)

out[onwrist] = state

df["state"] = out
df["sleep"] = np.where(out == 4,0,out)
print(nights)

In [ ]:
fig = actigraphy_single_plot_actogram(df, ["activity", "int_temp_","ext_temp_","sleep","offwrist"], [False, True, True, True, True], 12, dt = "datetime")
fig.show()